In [19]:
import pandas as pd
import numpy as np

### Распакуем данные в отдельную папку

In [3]:
import os
import zipfile

# Path to the ZIP file
zip_filepath = './hse-rec-sys-challenge-2024.zip'

# Destination folder where the extracted files will be placed
destination_folder = 'data'

# Ensure that the destination folder exists
os.makedirs(destination_folder, exist_ok=True)

# Unzip the file to the specified folder
with zipfile.ZipFile(zip_filepath, 'r') as zf:
    zf.extractall(path=destination_folder)

### Посмотрим на содержимое

In [9]:
events_df = pd.read_csv('./data/events.csv')
events_df.head()

,user_id,item_id,rating,timestamp
0,0,1505,4,0
1,0,3669,3,1
2,0,584,4,2
3,0,3390,3,3
4,0,2885,4,4


In [12]:
events_df[events_df['user_id']==0]

,user_id,item_id,rating,timestamp
0,0,1505,4,0
1,0,3669,3,1
2,0,584,4,2
3,0,3390,3,3
4,0,2885,4,4
...,...,...,...,...
282,0,1124,3,321
283,0,1809,4,322
284,0,3602,4,323
285,0,2924,3,325


In [13]:
item_features_df = pd.read_csv('./data/item_features.csv')
item_features_df.head()

,item_id,genre_0,genre_1,genre_2,genre_3,genre_4,genre_5,genre_6,genre_7,genre_8,genre_9,genre_10,genre_11,genre_12,genre_13,genre_14,genre_15,genre_16,genre_17
0,0,0,1,0,1,1,0,0,0,1,0,0,0,0,1,0,0,0,0
1,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
2,2,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
3,3,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0
4,4,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0


In [16]:
user_features_df = pd.read_csv('./data/user_features.csv')
user_features_df.head()

,user_id,gender,age
0,4855,F,1
1,4065,M,56
2,3331,M,25
3,5373,M,45
4,2032,M,25


In [14]:
submission_sample_df = pd.read_csv('./data/submission_sample.csv')
submission_sample_df.head()

,user_id,item_id
0,0,0 1 2 3 4 5 6 7 8 9
1,1,0 1 2 3 4 5 6 7 8 9
2,2,0 1 2 3 4 5 6 7 8 9
3,3,0 1 2 3 4 5 6 7 8 9
4,4,0 1 2 3 4 5 6 7 8 9


In [15]:
submission_sample_df.item_id

0       0 1 2 3 4 5 6 7 8 9
1       0 1 2 3 4 5 6 7 8 9
2       0 1 2 3 4 5 6 7 8 9
3       0 1 2 3 4 5 6 7 8 9
4       0 1 2 3 4 5 6 7 8 9
               ...         
6035    0 1 2 3 4 5 6 7 8 9
6036    0 1 2 3 4 5 6 7 8 9
6037    0 1 2 3 4 5 6 7 8 9
6038    0 1 2 3 4 5 6 7 8 9
6039    0 1 2 3 4 5 6 7 8 9
Name: item_id, Length: 6040, dtype: object

### Разобьем выборку на тренировочную, валидационнную и тестовую

In [28]:
import pandas as pd

def split_data_by_user(df, test_size=1, val_size=1):
    """
    Функция разделения данных на обучающую, валидационную и тестовую выборки.
    
    :param df: DataFrame с данными
    :param test_size: размер тестового набора (по умолчанию 1)
    :param val_size: размер валидационного набора (по умолчанию 1)
    :return: три DataFrame: train_df, val_df, test_df
    """
    # Группируем данные по каждому пользователю
    grouped = df.groupby('user_id')
    
    # Список для хранения индексов каждой группы
    train_indices = []
    val_indices = []
    test_indices = []
    
    # Проходимся по каждой группе
    for _, group in grouped:
        # Сортируем группу по timestamp
        sorted_group = group.sort_values(by='timestamp', ascending=False)
        
        # Получаем индексы для каждой выборки
        test_idx = sorted_group.index[:test_size]
        val_idx = sorted_group.index[test_size:test_size + val_size]
        train_idx = sorted_group.index[test_size + val_size:]
        
        # Добавляем индексы в соответствующие списки
        train_indices.extend(train_idx)
        val_indices.extend(val_idx)
        test_indices.extend(test_idx)
    
    # Формируем DataFrames для каждой выборки
    train_df = df.loc[train_indices].copy()
    val_df = df.loc[val_indices].copy()
    test_df = df.loc[test_indices].copy()
    
    return train_df, val_df, test_df

# Пример использования функции
train_df, val_df, test_df = split_data_by_user(events_df)

print("Train shape:", train_df.shape)
print("Val shape:", val_df.shape)
print("Test shape:", test_df.shape)

Train shape: (882069, 4)
Val shape: (6040, 4)
Test shape: (6040, 4)


### Сформируем матрицы 

размера (n_users, n_items) для обучающего и тестового наборов таким образом, чтобы элемент в ячейке [i, j] отражал оценку i-го пользователя j-му фильму 

https://makesomecode.me/2018/09/movie-recommendation-system/

In [17]:
n_users = user_features_df['user_id'].nunique()
n_items = item_features_df['item_id'].nunique()

n_users, n_items

(6040, 3706)

In [21]:
data_matrix = np.zeros((n_users, n_items))
for line in events_df.itertuples():
    data_matrix[line[1], line[2]] = line[3]
    

In [22]:
data_matrix

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 3.],
       [0., 0., 0., ..., 0., 0., 2.]])

In [25]:
from  sklearn.metrics.pairwise import pairwise_distances

# считаем косинусное расстояние для пользователей и фильмов 

# (построчно и поколоночно соотвественно).

user_similarity = pairwise_distances(data_matrix, metric='cosine')
item_similarity = pairwise_distances(data_matrix.T, metric='cosine')